### This notebook is intended to predict sql injection attacks
    Total data  : 5234
    Benign data : 3072 + 744 = 3816
    SQLI Data   : 1128 + 290 = 1418
    
#### At the end of notebook are results of testing

In [128]:

# import required packages

import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [129]:
import keras

In [130]:
import nltk
import pandas as pd
import os

### File_name: git_foospidy_payloads_fuzzing_code_db_sqli.txt
##### Comments :  
1. removed \n at the end of each line
2.  replaced %20 by space
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [131]:
# preprocess sql data to have same format for all files

def clean_sqli_data(data):
    
    for i in range(len(data)):
        
        data[i]=data[i].replace('\n', '')
        data[i]=data[i].replace('%20', ' ')
        data[i]=data[i].replace('=', ' = ')
        data[i]=data[i].replace('((', ' (( ')
        data[i]=data[i].replace('))', ' )) ')
        data[i]=data[i].replace('(', ' ( ')
        data[i]=data[i].replace(')', ' ) ')
    
    return data

In [132]:
path='./data/collection/git_foospidy_payloads_fuzzing_code_db_sqli.txt'

In [133]:
# read data from file

sql_lines_fuzzing=[]
f = open(path, "r")
for x in f:
    sql_lines_fuzzing.append(x)

In [134]:
sql_lines_fuzzing=clean_sqli_data(sql_lines_fuzzing) 

In [135]:
sql_lines_fuzzing[:15]

["'sqlvuln",
 "'+sqlvuln",
 'sqlvuln;',
 ' ( sqlvuln ) ',
 "a' or 1 = 1--",
 '"a"" or 1 = 1--"',
 ' or a  =  a',
 "a' or 'a'  =  'a",
 '1 or 1 = 1',
 "a' waitfor delay '0:0:10'--",
 "1 waitfor delay '0:0:10'--",
 'declare @q nvarchar  ( 4000 )  select @q  = ',
 '0x770061006900740066006F0072002000640065006C00610079002000270030003A0030003A',
 '0',
 '031003000270000']

### File_Name: git_foospidy_payloads_other_sqli_camoufl4g3.txt
##### Comments : ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [136]:
path='./data/collection/git_foospidy_payloads_other_sqli_camoufl4g3.txt'

In [137]:
# read data from file

sql_lines_camoufl4g3=[]
f = open(path, "r")
for x in f:
    sql_lines_camoufl4g3.append(x)

In [138]:
sql_lines_camoufl4g3[:15] # data before cleaning

["'-'\n",
 "' '\n",
 "'&'\n",
 "'^'\n",
 "'*'\n",
 "' or ''-'\n",
 "' or '' '\n",
 "' or ''&'\n",
 "' or ''^'\n",
 "' or ''*'\n",
 '"-"\n',
 '" "\n',
 '"&"\n',
 '"^"\n',
 '"*"\n']

In [139]:
sql_lines_camoufl4g3=clean_sqli_data(sql_lines_camoufl4g3)

In [140]:
sql_lines_camoufl4g3[:15]  # data after cleaning

["'-'",
 "' '",
 "'&'",
 "'^'",
 "'*'",
 "' or ''-'",
 "' or '' '",
 "' or ''&'",
 "' or ''^'",
 "' or ''*'",
 '"-"',
 '" "',
 '"&"',
 '"^"',
 '"*"']

### File_Name: git_foospidy_payloads_other_sqli_libinjection-bypasses.txt
##### Comments: Should Remove &()o1: from start of each sentence
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [141]:
path='./data/collection/git_foospidy_payloads_other_sqli_libinjection-bypasses.txt'

In [142]:
# read data from file

sql_lines_bypasses=[]
f = open(path, "r")
for x in f:
    sql_lines_bypasses.append(x)

In [143]:
sql_lines_bypasses[:5] # before cleaning

['&()o1: select * from users where id=1 or (\\)=1 union select 1,@@VERSION -- 1\n',
 '&(.)o: select * from users where id=1 or (\\.)=1 union select 1,@@VERSION -- 1\n',
 '&(1&1: select * from users where id=1 or (\\+)=1 or 1=1 -- 1\n',
 '&(1)o: select * from users where id=1 or (1)=1 union select 1,banner from v$version where rownum=1 -- 1\n',
 '&(1UE: select * from users where id=1 or (\\+)=1 union select 1,@@VERSION -- 1\n']

In [144]:
sql_lines_bypasses=clean_sqli_data(sql_lines_bypasses)

In [145]:
sql_lines_bypasses[:5]  # data after cleaning

['& (  ) o1: select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 '& ( . ) o: select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 '& ( 1&1: select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 '& ( 1 ) o: select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 '& ( 1UE: select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

In [146]:
# if don't want &(*)* sign in beginning of each sentence then run next code else don't

for i in range(len(sql_lines_bypasses)):
    sentence=sql_lines_bypasses[i]
    sql_lines_bypasses[i]=sentence.split(':')[1]
    

In [147]:
sql_lines_bypasses[:5]

[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

### File_Name: git_foospidy_payloads_owasp_sqli.txt
##### Comments: ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [148]:
path='./data/collection/git_foospidy_payloads_owasp_sqli.txt'

In [149]:
# read data from file

sql_lines_owasp=[]
f = open(path, "r")
for x in f:
    sql_lines_owasp.append(x)

In [150]:
sql_lines_owasp[0:15] # before cleaning

['a\n',
 "a' \n",
 "a' --\n",
 "a' or 1=1; --\n",
 '@\n',
 '?\n',
 "' and 1=0) union all\n",
 '? or 1=1 --\n',
 "x' and userid is NULL; --\n",
 "x' and email is NULL; --\n",
 "anything' or 'x'='x\n",
 "x' and 1=(select count(*) from tabname); --\n",
 "x' and members.email is NULL; --\n",
 "x' or full_name like '%bob%\n",
 '23 or 1=1; --\n']

In [151]:
sql_lines_owasp=clean_sqli_data(sql_lines_owasp)

In [152]:
sql_lines_owasp[:15] # after cleaning

['a',
 "a' ",
 "a' --",
 "a' or 1 = 1; --",
 '@',
 '?',
 "' and 1 = 0 )  union all",
 '? or 1 = 1 --',
 "x' and userid is NULL; --",
 "x' and email is NULL; --",
 "anything' or 'x' = 'x",
 "x' and 1 =  ( select count ( * )  from tabname ) ; --",
 "x' and members.email is NULL; --",
 "x' or full_name like '%bob%",
 '23 or 1 = 1; --']

### File_Name: git_seclist_fuzzing_sqli_Generic-SQLi.txt
##### comments: should we remove %20 by spaces because %20 equals to space
#### Source: https://github.com/danielmiessler/SecLists/tree/master/Fuzzing/SQLi

In [153]:
path='./data/collection/git_seclist_fuzzing_sqli_Generic-SQLi.txt'

In [154]:
# read data from file

sql_lines_Generic=[]
f = open(path, "r")
for x in f:
    sql_lines_Generic.append(x)

In [155]:
sql_lines_Generic[:15] # before cleaning

[")%20or%20('x'='x\n",
 '%20or%201=1\n',
 "; execute immediate 'sel' || 'ect us' || 'er'\n",
 'benchmark(10000000,MD5(1))#\n',
 'update\n',
 '";waitfor delay \'0:0:__TIME__\'--\n',
 '1) or pg_sleep(__TIME__)--\n',
 '||(elt(-3+5,bin(15),ord(10),hex(char(45))))\n',
 '"hi"") or (""a""=""a"\n',
 'delete\n',
 'like\n',
 '" or sleep(__TIME__)#\n',
 'pg_sleep(__TIME__)--\n',
 '*(|(objectclass=*))\n',
 'declare @q nvarchar (200) 0x730065006c00650063 ...\n']

In [156]:
sql_lines_Generic=clean_sqli_data(sql_lines_Generic)

In [157]:
sql_lines_Generic[:15] # after cleaning

[" )  or  ( 'x' = 'x",
 ' or 1 = 1',
 "; execute immediate 'sel' || 'ect us' || 'er'",
 'benchmark ( 10000000,MD5 ( 1  )  )  #',
 'update',
 '";waitfor delay \'0:0:__TIME__\'--',
 '1 )  or pg_sleep ( __TIME__ ) --',
 '|| ( elt ( -3+5,bin ( 15 ) ,ord ( 10 ) ,hex ( char ( 45  )  )    )  )  ',
 '"hi"" )  or  ( ""a"" = ""a"',
 'delete',
 'like',
 '" or sleep ( __TIME__ ) #',
 'pg_sleep ( __TIME__ ) --',
 '* ( | ( objectclass = *  )  )  ',
 'declare @q nvarchar  ( 200 )  0x730065006c00650063 ...']

### File_name: scottparker_ml_sqli_src__trainingdata_plain.txt
##### Comments: 
removed last 22 records containing URLs
#### Source: https://github.com/tungpv98/Detect-Sql-Injection-by-Machine-Learning/tree/ebeb3287931677a3a42f11a7a08dbc13e374ff05/Sql-Injection/source/trainingdata

In [158]:

# function to remove stopwords

stop_words = set(stopwords.words('english')) 

def fun_remove_stop_words(posts):

    filtered=''
    
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    
    return filtered

In [159]:
path='./data/collection/'
file="scottparker_ml_sqli_src__trainingdata_plain.txt"

In [160]:
#read benign data

df = pd.read_csv(os.path.join(path,file), sep='Aw3s0meSc0t7', names=['benign'], header=None, engine='python')

In [161]:
df.head()

,benign
0,Add plain text here
1,Ne te quaesiveris extra.
2,Man is his own star; and the soul that can
3,"Render an honest and a perfect man,"
4,"Commands all light, all influence, all fate;"


In [162]:
plain_text=df['benign'].values  # get sentences

In [163]:
plain_text[:5]

array(['Add plain text here', 'Ne te quaesiveris extra.',
       'Man is his own star; and the soul that can',
       'Render an honest and a perfect man,',
       'Commands all light, all influence, all fate;'], dtype=object)

In [164]:
plain_text=plain_text[:-22] # removed last 22 records that were urls

In [165]:
len(plain_text)

3672

In [166]:
# convert from list to string

data=''
for x in plain_text:
    data+=" " + x

In [167]:
type(data)

str

In [168]:
data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')              # split sentences

In [169]:
# seperate words inside tags

for i in range(len(data)):
    data[i]=data[i].replace('<', ' <')
    data[i]=data[i].replace('>', '> ')
    data[i]=data[i].replace('=', ' = ')

In [170]:
data[:5]

['  Add plain text Ne te quaesiveris extra',
 ' Man star; soul Render honest perfect man, Commands light, influence, fate; Nothing falls early late',
 ' Our acts angels are, good ill, Our fatal shadows walk us still',
 ' Epilogue Beaumont Fletcher’s Honest Man’s Fortune Cast bantling rocks, Suckle she-wolf’s teat; Wintered hawk fox, Power speed hands feet',
 ' I read day verses written eminent painter original conventional']

Statistics

In [171]:
print("Benign records: %2i" %len(data))

Benign records: 3072


In [172]:
# read self created benign data

path='./data/collection/benign_for_training.txt'
benign_data=[]
f = open(path, "r")
for x in f:
    benign_data.append(x)


In [173]:
# read self created sqli data

path='./data/collection/sqli_for_training.txt'
sqli_data=[]
f = open(path, "r")
for x in f:
    sqli_data.append(x)


In [174]:
len(benign_data)

442

In [175]:
benign_sentence=[]
for i in benign_data:
    sentences=i.split('.')
    
    for sentence in sentences:
        benign_sentence.append(sentence)

In [176]:
len(benign_sentence)

744

In [177]:
len(sqli_data)

290

In [178]:
print(f"SQL fuzzing : {len(sql_lines_fuzzing)}  camoufl4gs : {len(sql_lines_camoufl4g3)} parsed : {len(sql_lines_bypasses)} owasp : {len(sql_lines_owasp)} generic : {len(sql_lines_Generic)} \n total sql injection data : {len(sql_lines_Generic)+len(sql_lines_owasp)+len(sql_lines_bypasses)+len(sql_lines_camoufl4g3)+len(sql_lines_fuzzing)} ")

SQL fuzzing : 142  camoufl4gs : 77 parsed : 474 owasp : 167 generic : 268 
 total sql injection data : 1128 


#### combination of all sqli attacks

In [179]:
all_sqli_sentence=sql_lines_owasp+sql_lines_bypasses+sql_lines_camoufl4g3+sql_lines_fuzzing+sql_lines_Generic

In [180]:
len(all_sqli_sentence)

1128

In [181]:
# replace numeric values by a keyword 'numeric'
def optional_numeric_to_numeric(all_sqli_sentence):
    
    for i in range(len(all_sqli_sentence)):
        
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1 ', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 1', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("'1 ", "'numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 1'", " numeric'")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1,', 'numeric,')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("1\ ", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("‘1", '‘numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 2 ", " numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3--', ' numeric--')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 4 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 5 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 6 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 7 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1234', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("22", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 200 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("23 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('"1', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1"', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("7659", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 37 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 45 ", ' numeric ')
    
    return all_sqli_sentence


In [55]:
all_sqli_sentence

['a',
 "a' ",
 "a' --",
 "a' or 1 = 1; --",
 '@',
 '?',
 "' and 1 = 0 )  union all",
 '? or 1 = 1 --',
 "x' and userid is NULL; --",
 "x' and email is NULL; --",
 "anything' or 'x' = 'x",
 "x' and 1 =  ( select count ( * )  from tabname ) ; --",
 "x' and members.email is NULL; --",
 "x' or full_name like '%bob%",
 '23 or 1 = 1; --',
 "'; exec master..xp_cmdshell 'ping 172.10.1.255'--",
 'a',
 '1 or 1 = 1',
 "1' or '1' = '1",
 "1 and user_name (  )   =  'dbo'",
 '1',
 "1'1",
 '1 exec sp_  ( or exec xp_ ) ',
 '1 and 1 = 1',
 "1' and 1 =  ( select count ( * )  from tablenames ) ; --",
 '1',
 "1 and user_name (  )   =  'dbo'",
 "\\'; desc users; --",
 "1\\'1",
 "1' and non_existant_table  =  '1",
 "' or username is not NULL or username  =  '",
 "1 and ascii ( lower ( substring  (  (  select top 1 name from sysobjects where xtype = 'u' ) , 1, 1  )  )   )  > 116",
 "1 union all select 1,2,3,4,5,6,name from sysobjects where xtype  =  'u' --",
 '1 uni/**/on select all from where',
 '’ or ‘1’ =

#### Combination of benign an attack data

In [182]:
import pandas as pd

In [183]:
# give labels to sql data

values=[]
for i in all_sqli_sentence:
    values.append((i,1))

In [184]:
# give labels to benign data

for i in data:
    values.append((i,0))

In [185]:
len(all_sqli_sentence)+len(data)

4200

In [186]:
len(values)

4200

In [187]:
for i in benign_sentence:
    values.append((i,0))

In [188]:
len(values)

4944

In [189]:
for i in sqli_data:
    values.append((i,1))

In [190]:
len(values)

5234

In [191]:
values[1]

("a' ", 1)

In [192]:
# convert to dataframe

df=pd.DataFrame(values,columns=['Sentence','Label'])



In [193]:
df.head()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


### Save data as csv

In [194]:
df.to_csv('sqli.csv', index=False, encoding='utf-16')

In [195]:
df=pd.read_csv('sqli.csv',encoding='utf-16')

In [196]:
# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [197]:
posts.shape

(5234, 4096)

In [198]:
posts.shape=(5234,64,64,1)

In [199]:
posts.shape

(5234, 64, 64, 1)

In [200]:
X=posts

In [201]:
y=df['Label']

In [202]:
from sklearn.model_selection import train_test_split

In [203]:
# split train test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [204]:
X_train.shape

(4187, 64, 64, 1)

In [205]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [206]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [207]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)             

In [118]:
# import tensorflow as tf

# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('acc')>0.98):
#             print( "Reached 99.8% accuracy so cancelling training!")
#             self.model.stop_training=True
# callbacks = myCallback()

In [208]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128,)


Train on 4187 samples, validate on 1047 samples
Epoch 1/10
4187/4187 [==============================] - 26s 6ms/sample - loss: 0.3328 - accuracy: 0.8319 - val_loss: 0.2445 - val_accuracy: 0.8921
Epoch 2/10
4187/4187 [==============================] - 26s 6ms/sample - loss: 0.1888 - accuracy: 0.9033 - val_loss: 0.2084 - val_accuracy: 0.9026
Epoch 3/10
4187/4187 [==============================] - 26s 6ms/sample - loss: 0.1585 - accuracy: 0.9174 - val_loss: 0.1901 - val_accuracy: 0.9150
Epoch 4/10
4187/4187 [==============================] - 25s 6ms/sample - loss: 0.1404 - accuracy: 0.9257 - val_loss: 0.2021 - val_accuracy: 0.9226
Epoch 5/10
4187/4187 [==============================] - 27s 6ms/sample - loss: 0.1282 - accuracy: 0.9307 - val_loss: 0.1822 - val_accuracy: 0.9322
Epoch 6/10
4187/4187 [==============================] - 27s 6ms/sample - loss: 0.1062 - accuracy: 0.9424 - val_loss: 0.1860 - val_accuracy: 0.9312
Epoch 7/10
4187/4187 [==============================] - 29s 7ms/sample

In [209]:
pred=model.predict(X_test)

In [210]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [211]:
from sklearn.metrics import accuracy_score

In [212]:
accuracy_score(y_test,pred)

0.9398280802292264

In [213]:
for i,j in zip(y_test,pred):
    print(i==j)

[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[False]
[False]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]


#### Saving Model

In [214]:
from keras.models import load_model
import pickle

model.save('my_model_cnn.h5')
with open('vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)


### Confusion Matrix
#### Assumption:
1. SQLI attack type is positive

In [215]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [216]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [217]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [218]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [219]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [220]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9398280802292264 
 Precision : 0.8419540229885057 
 Recall : 0.973421926910299


In [221]:

from sklearn.metrics import precision_score
precision_score(y_test, pred)

0.8419540229885057

In [222]:
from sklearn.metrics import recall_score
recall_score(y_test, pred)


0.973421926910299

### Classifying user data

     This cell is also in file user_Data_predict.py No need to run this cell

In [223]:

import keras
from keras.models import load_model
import pickle


mymodel = tf.keras.models.load_model('my_model_cnn.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()
    
    input_val.shape=(1,64,64,1)

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

 



In [224]:
# check for sql injection cheat sheet





predict_sqli_attack()

Give me some data to work on : AND (SELECT version()) LIKE "5%"--
ALERT :::: This can be SQL injection
Give me some data to work on : TABLE_NAME="table1" LIMIT 0,1)))--
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT * from table where id =1 union select 1,2,3--
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT * FROM items WHERE 'a'='a'
ALERT :::: This can be SQL injection
Give me some data to work on : id=1' waitfor delay '00:00:10'--
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT title, description, body FROM items WHERE ID = 2
ALERT :::: This can be SQL injection
Give me some data to work on : SELECT BENCHMARK(1000000,MD5('A')); id=1 OR SLEEP(25)=0 LIMIT 1-- id=1) OR SLEEP(25)=0 LIMIT 1-- id=1' OR SLEEP(25)=0
ALERT :::: This can be SQL injection
Give me some data to work on : LIMIT 1-- id=1') OR SLEEP(25)=0 LIMIT 1-- id=1)) OR SLEEP(25)=0 LIMIT 1-- id=SELECT SLEEP(25)--
ALERT :::: This can be SQL inject

In [3]:
# check for normal data





predict_sqli_attack()

Give me some data to work on : wait for me
It seems to be safe
Give me some data to work on : a quick brown fox
It seems to be safe
Give me some data to work on : jumps over the alxy dog
It seems to be safe
Give me some data to work on : lazy dog correct
It seems to be safe
Give me some data to work on : And what time will I be I would like an evening flight on the return flight in 4 days
It seems to be safe
Give me some data to work on : that is nonstop the flight duration is 2 hours and 28 minutes and is priced at $337
It seems to be safe
Give me some data to work on : That sounds very good
It seems to be safe
Give me some data to work on : I would like to fly on Delt
It seems to be safe
Give me some data to work on : select from the following menu
It seems to be safe
Give me some data to work on : his menu is to select some food from it
It seems to be safe
Give me some data to work on : Does that complete your order
It seems to be safe
Give me some data to work on : I'd like to sele

In [225]:
predict_sqli_attack()


Give me some data to work on : i select this sentence but this should not be injection it is like injection but from my point of view it should not be even if i include union in it like before
It seems to be safe
Give me some data to work on : 0
 Good Bye 
